# Elementary Effects for the BTD Model

## Setup packages.

In [ ]:
require(data.table)
require(magrittr)
require(sensitivity)

require(ggplot2)

## Design experiment.

### Load input ranges.

In [2]:
z.ranges <- fread("input-ranges.tsv")
z.ranges %>% dim

[1] 82  9

In [3]:
z.ranges %>% summary

   Variable            Units               Type           Model Minimum    
 Length:82          Length:82          Length:82          Min.   : -10.00  
 Class :character   Class :character   Class :character   1st Qu.:   0.00  
 Mode  :character   Mode  :character   Mode  :character   Median :   0.00  
                                                          Mean   :  24.48  
                                                          3rd Qu.:   0.00  
                                                          Max.   :2015.00  
 Model Maximum      Model Default       Sensitivity Minimum Sensitivity Maximum
 Length:82          Min.   :0.000e+00   Min.   :        0   Min.   :0.000e+00  
 Class :character   1st Qu.:0.000e+00   1st Qu.:        0   1st Qu.:1.000e+00  
 Mode  :character   Median :1.000e+00   Median :        1   Median :5.000e+00  
                    Mean   :1.572e+07   Mean   :  9432283   Mean   :3.735e+07  
                    3rd Qu.:9.750e+02   3rd Qu.:      585   3rd Qu.:

### One-at a time experiment with 500 repetitions, à la Morris.

In [4]:
z.design <- morris(
    NULL,
    factors = z.ranges$Variable,
    r = 500,
    design = list(
        type = "oat",
        levels = mapply(function(t, x0, x1) {
            if (t == "Integer")
                x1 - x0 + 1
            else if (t == "Boolean")
                2
            else
                5
        }, z.ranges$Type, z.ranges$`Sensitivity Minimum`, z.ranges$`Sensitivity Maximum`),
        grid.jump = 1
    )
)
z.design$X %>% dim

[1] 41500    82

In [5]:
write.table(z.design$X, file = "design.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

### Relate the design to the model's variables.

In [6]:
z.inputs <- cbind(
    Run = 1:(dim(z.design$X)[1]),
    data.table(
        sweep(
            sweep(z.design$X, MARGIN = 2, z.ranges$`Sensitivity Maximum` - z.ranges$`Sensitivity Minimum`, `*`),
            MARGIN = 2,
            z.ranges$`Sensitivity Minimum`,
            `+`
        )
    )
)
z.inputs %>% summary

      Run        advertising budget advertising start time
 Min.   :    1   Min.   :300000     Min.   :2015          
 1st Qu.:10376   1st Qu.:425000     1st Qu.:2024          
 Median :20750   Median :550000     Median :2032          
 Mean   :20750   Mean   :553389     Mean   :2032          
 3rd Qu.:31125   3rd Qu.:675000     3rd Qu.:2041          
 Max.   :41500   Max.   :800000     Max.   :2050          
 aversion to NPV deviation base external investor ask rate
 Min.   :0.1200            Min.   : 4.800                 
 1st Qu.:0.1700            1st Qu.: 6.800                 
 Median :0.2200            Median : 8.800                 
 Mean   :0.2241            Mean   : 8.695                 
 3rd Qu.:0.2700            3rd Qu.:10.800                 
 Max.   :0.3200            Max.   :12.800                 
 bioproduct long term price bioproduct offtake agreement
 Min.   :3000               Min.   :0.3000              
 1st Qu.:4250               1st Qu.:0.4250              
 Me

In [7]:
write.table(z.inputs, file="inputs.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

## Analyze results.

### Read design.

In [ ]:
z.design <- fread("design.tsv")
z.design %>% dim

### Read inputs.

In [ ]:
z.inputs <- fread("inputs.tsv")
z.inputs %>% dim

### Read outputs.

In [ ]:
z.outputs <- fread("outputs.tsv")
z.outputs[Time == 2050] %>% summary

### Clean up outputs by filling in missing results with zero cumulative production.

In [ ]:
z.outputs.clean <- z.outputs[`Time` == 2050, .(Run, `Cumulative Production`)]
z.outputs.clean <- rbind(
    z.outputs.clean,
    data.table(Run = setdiff(1:5900, z.outputs[`Time` == 2050, `Run`]), `Cumulative Production` = 0)
)[order(Run)]$`Cumulative Production`
z.outputs.clean %>% summary

### Define functions to compute elementary effects.

In [ ]:
ind.rep <- function(i, p) {
# indices of the points of the ith trajectory in the DoE
  (1 : (p + 1)) + (i - 1) * (p + 1)
}

ee.oat <- function(X, y) {
  # compute the elementary effects for a OAT design
  p <- ncol(X)
  r <- nrow(X) / (p + 1)
  
#  if(is(y,"numeric")){
  if(inherits(y, "numeric")){
    one_i_vector <- function(i){
      j <- ind.rep(i, p)
      j1 <- j[1 : p]
      j2 <- j[2 : (p + 1)]
      # return((y[j2] - y[j1]) / rowSums(X[j2,] - X[j1,]))
      return(solve(X[j2,] - X[j1,], y[j2] - y[j1]))
    }
    ee <- vapply(1:r, one_i_vector, FUN.VALUE = numeric(p))
    ee <- t(ee)
    # "ee" is now a (r times p)-matrix.
#  } else if(is(y,"matrix")){
  } else if(inherits(y, "matrix")){
    one_i_matrix <- function(i){
      j <- ind.rep(i, p)
      j1 <- j[1 : p]
      j2 <- j[2 : (p + 1)]
      return(solve(X[j2,] - X[j1,], 
                   y[j2, , drop = FALSE] - y[j1, , drop = FALSE]))
    }
    ee <- vapply(1:r, one_i_matrix, 
                 FUN.VALUE = matrix(0, nrow = p, ncol = dim(y)[2]))
    # Special case handling for p == 1 and ncol(y) == 1 (in this case, "ee" is
    # a vector of length "r"):
    if(p == 1 && dim(y)[2] == 1){
      ee <- array(ee, dim = c(r, 1, 1))
    }
    # Transpose "ee" (an array of dimensions c(p, ncol(y), r)) to an array of
    # dimensions c(r, p, ncol(y)) (for better consistency with the standard 
    # case that "class(y) == "numeric""):
    ee <- aperm(ee, perm = c(3, 1, 2))
#  } else if(is(y,"array")){
  } else if(inherits(y, "array")){
    one_i_array <- function(i){
      j <- ind.rep(i, p)
      j1 <- j[1 : p]
      j2 <- j[2 : (p + 1)]
      ee_per_3rd_dim <- sapply(1:(dim(y)[3]), function(idx_3rd_dim){
        y_j2_matrix <- y[j2, , idx_3rd_dim]
        y_j1_matrix <- y[j1, , idx_3rd_dim]
        # Here, the result of "solve(...)" is a (p times dim(y)[2])-matrix or
        # a vector of length dim(y)[2] (if p == 1):
        solve(X[j2,] - X[j1,], y_j2_matrix - y_j1_matrix)
      }, simplify = "array")
      if(dim(y)[2] == 1){
        # Correction needed if dim(y)[2] == 1, so "y_j2_matrix" and
        # "y_j1_matrix" have been dropped to matrices (or even vectors, if also
        # p == 1):
        ee_per_3rd_dim <- array(ee_per_3rd_dim, 
                                dim = c(p, dim(y)[2], dim(y)[3]))
      } else if(p == 1){
        # Correction needed if p == 1 (and dim(y)[2] > 1), so "y_j2_matrix" and
        # "y_j1_matrix" have been dropped to matrices:
        ee_per_3rd_dim <- array(ee_per_3rd_dim, 
                                dim = c(1, dim(y)[2], dim(y)[3]))
      }
      # "ee_per_3rd_dim" is now an array of dimensions 
      # c(p, dim(y)[2], dim(y)[3]). Assign the corresponding names for the 
      # third dimension:
      if(is.null(dimnames(ee_per_3rd_dim))){
        dimnames(ee_per_3rd_dim) <- dimnames(y)
      } else{
        dimnames(ee_per_3rd_dim)[[3]] <- dimnames(y)[[3]]
      }
      return(ee_per_3rd_dim)
    }
    ee <- sapply(1:r, one_i_array, simplify = "array")
    # Special case handling if "ee" has been dropped to a vector:
#    if(is(ee,"numeric")){
    if (inherits(ee, "numeric")){
      ee <- array(ee, dim = c(p, dim(y)[2], dim(y)[3], r))
      dimnames(ee) <- list(NULL, dimnames(y)[[2]], dimnames(y)[[3]], NULL)
    }
    # "ee" is an array of dimensions c(p, dim(y)[2], dim(y)[3], r), so it is
    # transposed to an array of dimensions c(r, p, dim(y)[2], dim(y)[3]):
    ee <- aperm(ee, perm = c(4, 1, 2, 3))
  }
  return(ee)
}

### Compute the elementary effects.

In [ ]:
z.ee <- ee.oat(z.design, z.outputs.clean)
z.ee %>% dim

### Compute mu, mu-start, and sigma.

In [ ]:
z.mu <- apply(z.ee, 2, mean)
z.mustar <- apply(z.ee, 2, function(x) mean(abs(x)))
z.sigma <- apply(z.ee, 2, sd)
z.result <- data.table(
    variable = names(z.mu),
    mu       = z.mu       ,
    mustar   = z.mustar   ,
    sigma    = z.sigma
)

### Sort the results in decreasing order of influence.

Interpretations:
*   mu: influence of variable
*   mustar: influence of variable, accounting for non-monoticity
*   sigma: non-linear and interaction effects for variable

In [ ]:
z.result[, `:=`(`mu rank` = frank(-mu), `mustar rank` = frank(-mustar), `sigma rank` = frank(-sigma))]
z.result[order(-mustar)]

### Plot mu vs sigma.

In [ ]:
ggplot(z.result, aes(x = mu, y = sigma, color = mustar, label = variable)) +
    geom_point() +
    geom_text(size = 2, hjust = 0, vjust = 0)